Блокнот создания индекса с векторным поиском на основе идекса option-index

Если индекс уже существует, то он не перезапишется, а увеличится

In [ ]:
OLD_INDEX = 'option-index'
NEW_INDEX = 'vector-index'

In [1]:
import requests

from tqdm import tqdm
from vector_search import HybridOpenSearch
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vs = HybridOpenSearch(
    opensearch_url='http://localhost:9200',
    index_name=NEW_INDEX,
    embedding_function=embedder,
    search={
        "k": 10,
        "space_type": "cosinesimil",
        "weights": [0.8, 0.2]
    }
)

In [ ]:
body = {
    'size': 1,
    'from': 0,
    'query' : {
        'match_all' : {}
    }
}
count = requests.get(url=f'http://localhost:9200/{OLD_INDEX}/_count').json()['count']

def get_item_from_old_index():
    for i in tqdm(range(count)):
        body['from'] = i
        response = requests.get(
            url=f'http://localhost:9200/{OLD_INDEX}/_search',
            json=body
        )
        yield response.json()

In [7]:
iterator = get_item_from_old_index()

In [ ]:
while (item:=next(iterator, None)) is not None:
    text = item['hits']['hits'][0]['_source']['option_all_text']
    new_item = item['hits']['hits'][0]['_source']
    vs.add_texts(
        texts=[text],
        metadatas=[new_item]
    )

100%|██████████| 810/810 [03:23<00:00,  3.97it/s]
